In [1]:
#!pip3 install --upgrade gym
#!pip3 install --upgrade box2d-py
#!pip3 install opencv-python
#from gym.envs.box2d.car_dynamics import Car

array([0.2187804 , 0.7549155 , 0.46578676], dtype=float32)

In [4]:
#env=gym.make('CarRacing-v0')
#env.action_space
#env.action_space.sample()

In [5]:

envc.reset()
nst,rw,ter,info=envc.step([0,1,0])

Track generation: 1072..1344 -> 272-tiles track


In [4]:
def accl_to_gas(accl,vel):
    SIZE = 0.02
    ENGINE_POWER= 100000000*SIZE*SIZE
    WHEEL_MOMENT_OF_INERTIA = 4000*SIZE*SIZE
    WHEEL_R  = 27 
    wheel_rad=WHEEL_R*SIZE
    mass=(2*WHEEL_MOMENT_OF_INERTIA)/wheel_rad
    mass=4*mass
    #mass=.25
    power=mass*accl*(vel+accl/2)
    gas=power/ENGINE_POWER
    return gas


In [5]:
gas=accl_to_gas(1,0)
SIZE = 0.02
ENGINE_POWER= 100000000*SIZE*SIZE
gas*ENGINE_POWER

11.851851851851851

In [19]:
def accl_to_gas(accl,vel):
    SIZE = 0.02
    ENGINE_POWER= 100000000*SIZE*SIZE
    WHEEL_MOMENT_OF_INERTIA = 4000*SIZE*SIZE
    WHEEL_R  = 27 
    wheel_rad=WHEEL_R*SIZE
    mass=(2*WHEEL_MOMENT_OF_INERTIA)/wheel_rad
    #mass=.25
    power=4*mass*accl*vel
    gas=power/ENGINE_POWER
    return gas

def env_step(env,action):
    plt.cla()
    frame=np.zeros((100,200,3),dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    nst,rw,ter,info=envc.step(action) # take a random action
    frame[0:96,0:96,:]=np.array(nst)
    
    pos=envc.car.hull.position
    vel=np.sqrt(envc.car.hull.linearVelocity[0]**2+envc.car.hull.linearVelocity[1]**2)
    ang=envc.car.hull.angle
    
    img = cv2.putText(np.array(frame), 'vel %f'%vel, (100,10), font,.3, (255,255,0),1, cv2.LINE_AA)
    img = cv2.putText(np.array(img), 'ang %f'%ang, (100,20), font,.3, (255,0,255),1, cv2.LINE_AA)
    image = cv2.putText(np.array(img), 'x,y %f,%f'%(pos[0],pos[1]), (100,30), font,.2, (255,0,255),1, cv2.LINE_AA)
        
    plt.imshow(image)
    plt.show()
    plt.pause(0.0001)
        
    return [pos[0],pos[1],vel,ang],rw,ter,info
#envc.close()

In [41]:
envc.close()

In [5]:
%matplotlib tk
import gym
from matplotlib import pyplot as plt
import cv2
import numpy as np
from car_env import CarRacing
import time

envc=CarRacing()
def env_reset(envc):
    import pickle
    envc.reset()
    save=False
    if save==True:
        f=open('path-1','wb')
        pickle.dump(envc.track,f)
        f.close()
    f=open('path-1','rb')
    track=pickle.load(f)
    f.close()
    envc.track=track
    
    points=[[x,y] for a,b,x,y in envc.track]
    points=np.array(points)
    plt.plot(points[:,0],points[:,1])
    plt.show()
    return env_step(envc,[0,0,0])
    

for i in range(100):
    st,rw,ter,info=env_step(envc,[0,1,0])
    time.sleep(0.1)
    #print(st)
envc.close()

Track generation: 1041..1305 -> 264-tiles track


In [16]:
plt.plot(points[:,0]+150,points[:,1]+150)
plt.show()

In [ ]:
cv2.imwrite(data)

In [8]:
#https://navoshta.com/detecting-road-features/
#https://towardsdatascience.com/tutorial-build-a-lane-detector-679fd8953132
#https://towardsdatascience.com/finding-lane-lines-simple-pipeline-for-lane-detection-d02b62e7572b

In [38]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt

In [39]:
class image_to_traj():
    def __init__(self,N=10):
        self.N=N
    #@property
    def run(self,image):
        canny = self.do_canny(image)
        segment = self.do_segment(canny)
        ret= self.get_points(segment)
        return ret
        #hough = cv2.HoughLinesP(segment, 1, np.pi / 180, 10, np.array([]), minLineLength = 5, maxLineGap = 5)
    def do_canny(self,image):
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        canny = cv2.Canny(blur, 50, 150)
        return canny

    def do_segment(self,img):
        height = img.shape[0]
        polygons = np.array([
                                [(0, 80), (96, 80), (0, 0),(96,0)]
                            ])
        mask = np.zeros_like(img)
        cv2.fillPoly(mask, polygons, 255)
        segment = cv2.bitwise_and(img, mask)
        return segment

    def get_points(self,segment):
        crops=[]
        cxs=[]
        _cys=[]
        cys=[]
        offset=self.N//2
        height=segment.shape[1]
        for i in range(height//self.N):
            crop=segment[i*self.N:(i+1)*self.N,:]
            crops.append(crop)
            M=cv2.moments(crop)
            try:
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
            except:
                pass
                #cx=20
                #cy=N
            cxs.append(cx)
            _cys.append(cy)
            cys.append(offset+cy)
            #print(cx,cy)
            #print(crop)
            _crop=crop.copy()
            img = cv2.circle(_crop,(cx,cy), 2, (255,255,255), -1)
            #plt.imshow(img,cmap='gray',vmin=0,vmax=255)
            #plt.show()_
            offset+=self.N
            #print(i)
        return crops,cxs,_cys,cys

In [40]:
i=0
N=10
frame=cv2.imread('car_lane.jpg')
i2t=image_to_traj(N=10)
[crops,cxs,cys,hh]=i2t.run(frame)
#crops,cxs,cys=get_points(segment,N)
for x,y,crop in zip(cxs,cys,crops):
    img = cv2.circle(frame,(x,i+y), 2, (255,255,255), -1)
    print('x,y :',x,y+i,end=' ')
    i+=N
    
plt.imshow(img,cmap='gray',vmin=0,vmax=255)
plt.show()

x,y : 21 5 x,y : 32 14 x,y : 43 23 x,y : 47 33 x,y : 47 46 x,y : 47 54 x,y : 47 64 x,y : 47 74 x,y : 47 80 

In [71]:
# Draw the lines
if hough is not None:
        for i in range(0, len(hough)):
            l = hough[i][0]
            _ss=cv2.line(canny, (l[0], l[1]), (l[2], l[3]), (255,255,255), 1, cv.LINE_AA)

In [72]:
plt.imshow(nst)

In [73]:
#cv2.imwrite(nst,'car_lane.jpg')

TypeError: bad argument type for built-in operation

In [49]:
#len(hough)
_ss.shape

(96, 96)